In [1]:
import os
import time

import dask.dataframe as dd
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

### For plotting

In [68]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [163]:
cmap = plt.get_cmap('cubehelix', 5)
new_cmap = truncate_colormap(cmap, 0.2, 0.75)

In [176]:
color = []

for i in np.linspace(0, 1, 7):
    color.append(colors.rgb2hex(new_cmap(i)))

In [202]:
palette = sns.color_palette(color)
palette_two = sns.color_palette(color[1::3])
palette_three = sns.color_palette(color[1::2])
palette_four = sns.color_palette(color[0::2])

In [190]:
palette

[(0.08627450980392157, 0.3254901960784314, 0.2980392156862745),
 (0.08627450980392157, 0.3254901960784314, 0.2980392156862745),
 (0.08627450980392157, 0.3254901960784314, 0.2980392156862745),
 (0.6274509803921569, 0.4745098039215686, 0.28627450980392155),
 (0.6274509803921569, 0.4745098039215686, 0.28627450980392155),
 (0.7803921568627451, 0.7019607843137254, 0.9294117647058824),
 (0.7803921568627451, 0.7019607843137254, 0.9294117647058824)]

In [203]:
palette_three

[(0.08627450980392157, 0.3254901960784314, 0.2980392156862745),
 (0.6274509803921569, 0.4745098039215686, 0.28627450980392155),
 (0.7803921568627451, 0.7019607843137254, 0.9294117647058824)]

In [198]:
palette_four

[(0.08627450980392157, 0.3254901960784314, 0.2980392156862745),
 (0.08627450980392157, 0.3254901960784314, 0.2980392156862745),
 (0.6274509803921569, 0.4745098039215686, 0.28627450980392155),
 (0.7803921568627451, 0.7019607843137254, 0.9294117647058824)]

In [72]:
# Order of all possible bins for plotting
order = ['(0, 10]', '(10, 20]', '(20, 30]', '(30, 40]', '(40, 50]',
         '(50, 60]', '(60, 70]', '(70, 80]', '(80, 90]', '(90, 100]',
         '(100, 150]', '(150, 200]', '(200, 500]']

In [ ]:
# save_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/figures_for_2025_05_27/'

## Threshold

In [47]:
# Thresholds
thresholds = ['0.1', '0.2', '0.3', '0.4', 
              '0.5', '0.6', '0.7', '0.8', '0.9']

### HUC2_01, Northeast

In [130]:
## Setup
bwidth = 'mean'
bbin = 'Bin'
huc2 = '01'
data_path = '../../narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_' + \
            huc2 + '_2025_05_24_' + bwidth

In [131]:
# Get list of files    
files = []
for file in os.listdir(data_path):
    if not file.startswith('.'):
        files.append(os.path.join(data_path, file))

In [132]:
ddf_ne = dd.read_parquet(files[-1000:])

In [133]:
ddf_ne['0.1'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.1].count(axis=1)
ddf_ne['0.2'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.2].count(axis=1)
ddf_ne['0.3'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.3].count(axis=1)
ddf_ne['0.4'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.4].count(axis=1)
ddf_ne['0.5'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.5].count(axis=1)
ddf_ne['0.6'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.6].count(axis=1)
ddf_ne['0.7'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.7].count(axis=1)
ddf_ne['0.8'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.8].count(axis=1)
ddf_ne['0.9'] = ddf_ne.iloc[:,1:11][ddf_ne.iloc[:,1:11] > 0.9].count(axis=1)

In [134]:
ddf_ne = ddf_ne.drop(list(ddf_ne.columns[1:11]), axis=1)

In [135]:
ddf_ne_melted = ddf_ne.melt(id_vars=['NHDPlusID', 'Bin'],
                            value_vars=['0.1', '0.2', '0.3', '0.4', '0.5',
                                        '0.6', '0.7', '0.8', '0.9'],
                            var_name='threshold', value_name='coverage')

### troubleshoot

In [93]:
# test = ddf_ne.get_partition(0).compute()

In [94]:
# test = test.drop(test.columns[1:11], axis=1)

In [95]:
# test.melt(id_vars=['NHDPlusID'],
#           value_vars=['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9'],
#           var_name='threshold')

In [20]:
# ddf_ne_melted = ddf_ne_melted.compute().reset_index().drop(columns='index')

In [18]:
len(ddf_ne.NHDPlusID.unique())

102895

In [19]:
len(ddf_ne_melted.NHDPlusID.unique())

102895

In [15]:
# ne = ddf_ne_melted[ddf_ne_melted['coverage'] != 0]

In [16]:
# ne = ne[ne['threshold'] == thresholds[4]].reset_index().drop(columns='index')

In [17]:
# ne.compute()

In [36]:
ddf_ne = ddf_ne.drop(list(ddf_ne.columns[-9:]), axis=1)

In [37]:
temp = ddf_ne_melted.merge(right=ddf_ne, how='left', on='NHDPlusID').reset_index()

In [40]:
temp.compute().iloc[[0]].NHDPlusID

0    5.000200e+12
Name: NHDPlusID, dtype: float64

In [42]:
temp = temp.compute()

In [45]:
temp.iloc[0].NHDPlusID

np.float64(5000200000005.0)

In [56]:
temp[(temp.threshold == '0.5')].shape

(63345, 25)

In [57]:
temp[(temp.threshold == '0.5') & (temp.coverage != 0)].shape

(8719, 25)

In [ ]:
temp[(temp.threshold == '0.5') & (temp.coverage != 0) & ~(temp.swot_slope.isna())]

In [ ]:
zeros = temp[temp['coverage'] == 0]

In [ ]:
ne = temp[temp['coverage'] != 0]

In [ ]:
ne['coverage'] = ne['coverage']*10

In [ ]:
ne = ne[['threshold', 'coverage', 'Bin', 'alignment']]

In [ ]:
ne = ne[ne['threshold'] == thresholds[4]].reset_index().drop(columns='index')

In [ ]:
ne.columns

In [ ]:
ne = ne.merge(ddf_ne[['NHDPlusID', bbin]], how='left', on='NHDPlusID')

In [ ]:
ne = ne.drop(columns='NHDPlusID')

In [ ]:
ne.compute()

In [ ]:
# test = ne.sample(frac=0.00005)

In [ ]:
del ddf_ne, ddf_ne_melted, temp

In [ ]:
# testy = test.get_partition(0)

In [ ]:
# testy = testy.compute()

In [ ]:
# ddf_ne = ddf_ne.sample(frac=0.1)

In [ ]:
# ne = ddf_ne[ddf_ne['threshold'] == thresholds[8]].reset_index()#.drop(columns='index')

In [ ]:
# ne = ne[['num_wse']]

In [ ]:
# del ne

In [ ]:
# ne = ne.sample(frac=0.005)

In [ ]:
# unique_val = ddf_ne['Bin'].unique()

In [ ]:
# unique_val.compute()

### HUC2_08, Southeast

In [123]:
## Setup
bwidth = 'mean'
bbin = 'Bin'
huc2 = '08'
data_path = '../../narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_' + \
            huc2 + '_2025_05_24_' + bwidth

In [124]:
# Get list of files    
files = []
for file in os.listdir(data_path):
    if not file.startswith('.'):
        files.append(os.path.join(data_path, file))

In [125]:
ddf_se = dd.read_parquet(files[-1000:])

In [126]:
ddf_se['0.1'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.1].count(axis=1)
ddf_se['0.2'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.2].count(axis=1)
ddf_se['0.3'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.3].count(axis=1)
ddf_se['0.4'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.4].count(axis=1)
ddf_se['0.5'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.5].count(axis=1)
ddf_se['0.6'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.6].count(axis=1)
ddf_se['0.7'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.7].count(axis=1)
ddf_se['0.8'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.8].count(axis=1)
ddf_se['0.9'] = ddf_se.iloc[:,1:11][ddf_se.iloc[:,1:11] > 0.9].count(axis=1)

In [127]:
ddf_se = ddf_se.drop(list(ddf_se.columns[1:11]), axis=1)

In [128]:
ddf_se_melted = ddf_se.melt(id_vars=['NHDPlusID', 'Bin'],
                            value_vars=['0.1', '0.2', '0.3', '0.4', '0.5',
                                        '0.6', '0.7', '0.8', '0.9'],
                            var_name='threshold', value_name='coverage')

In [129]:
# ddf_se_melted = ddf_se_melted.compute().reset_index().drop(columns='index')

### HUC2_15, Southwest

In [136]:
## Setup
bwidth = 'mean'
bbin = 'Bin'
huc2 = '15'
data_path = '../../narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_' + \
            huc2 + '_2025_05_24_' + bwidth

In [137]:
# Get list of files    
files = []
for file in os.listdir(data_path):
    if not file.startswith('.'):
        files.append(os.path.join(data_path, file))

In [138]:
ddf_sw = dd.read_parquet(files[-1000:])

In [139]:
ddf_sw['0.1'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.1].count(axis=1)
ddf_sw['0.2'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.2].count(axis=1)
ddf_sw['0.3'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.3].count(axis=1)
ddf_sw['0.4'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.4].count(axis=1)
ddf_sw['0.5'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.5].count(axis=1)
ddf_sw['0.6'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.6].count(axis=1)
ddf_sw['0.7'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.7].count(axis=1)
ddf_sw['0.8'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.8].count(axis=1)
ddf_sw['0.9'] = ddf_sw.iloc[:,1:11][ddf_sw.iloc[:,1:11] > 0.9].count(axis=1)

In [140]:
ddf_sw = ddf_sw.drop(list(ddf_sw.columns[1:11]), axis=1)

In [141]:
ddf_sw_melted = ddf_sw.melt(id_vars=['NHDPlusID', 'Bin'],
                            value_vars=['0.1', '0.2', '0.3', '0.4', '0.5',
                                        '0.6', '0.7', '0.8', '0.9'],
                            var_name='threshold', value_name='coverage')

In [50]:
# ddf_sw_melted = ddf_sw_melted.compute().reset_index().drop(columns='index')

### HUC2_17, Northwest

In [142]:
## Setup
bwidth = 'mean'
bbin = 'Bin'
huc2 = '17'
data_path = '../../narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_' + \
            huc2 + '_2025_05_24_' + bwidth

In [143]:
# Get list of files    
files = []
for file in os.listdir(data_path):
    if not file.startswith('.'):
        files.append(os.path.join(data_path, file))

In [145]:
ddf_nw = dd.read_parquet(files[-1000:])

In [146]:
ddf_nw['0.1'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.1].count(axis=1)
ddf_nw['0.2'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.2].count(axis=1)
ddf_nw['0.3'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.3].count(axis=1)
ddf_nw['0.4'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.4].count(axis=1)
ddf_nw['0.5'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.5].count(axis=1)
ddf_nw['0.6'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.6].count(axis=1)
ddf_nw['0.7'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.7].count(axis=1)
ddf_nw['0.8'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.8].count(axis=1)
ddf_nw['0.9'] = ddf_nw.iloc[:,1:11][ddf_nw.iloc[:,1:11] > 0.9].count(axis=1)

In [147]:
ddf_nw = ddf_nw.drop(list(ddf_nw.columns[1:11]), axis=1)

In [148]:
ddf_nw_melted = ddf_nw.melt(id_vars=['NHDPlusID', 'Bin'],
                            value_vars=['0.1', '0.2', '0.3', '0.4', '0.5',
                                        '0.6', '0.7', '0.8', '0.9'],
                            var_name='threshold', value_name='coverage')

In [149]:
# ddf_nw_melted = ddf_nw_melted.compute().reset_index().drop(columns='index')

### Four

In [ ]:
# for i in range(len(thresholds)):
i = 7

# Get ne reaches at threshold
ne = ddf_ne_melted[ddf_ne_melted['threshold'] == thresholds[i]].reset_index()#.drop(columns='index')
# ne = ne.sample(frac=0.05)
ne = ne.compute()

zeros = ne[ne.coverage == 0].count()
reaches = ne.shape[0]
ne_zeros = round(((zeros['coverage']/reaches)*100), 1)

ne = ne[ne.coverage != 0]
if ne.shape[0] > 100000:
    ne = ne.sample(100000)
ne['huc2'] = '01'

# Get se reaches at threshold
se = ddf_se_melted[ddf_se_melted['threshold'] == thresholds[i]].reset_index().drop(columns='index')
# se = se.sample(frac=0.05)
se = se.compute()

zeros = se[se.coverage == 0].count()
reaches = se.shape[0]
se_zeros = round(((zeros['coverage']/reaches)*100), 1)

se = se[se.coverage != 0]
if se.shape[0] > 100000:
    se = se.sample(100000)
se['huc2'] = '08'

# Get sw reaches
sw = ddf_sw_melted[ddf_sw_melted['threshold'] == thresholds[i]].reset_index().drop(columns='index')
# sw = sw.sample(frac=0.05)
sw = sw.compute()

zeros = sw[sw.coverage == 0].count()
reaches = sw.shape[0]
sw_zeros = round(((zeros['coverage']/reaches)*100), 1)

sw = sw[sw.coverage != 0]
if sw.shape[0] > 100000:
    sw = sw.sample(100000)
sw['huc2'] = '15'

# Get nw reaches
nw = ddf_nw_melted[ddf_nw_melted['threshold'] == thresholds[i]].reset_index().drop(columns='index')
# nw = nw.sample(frac=0.05)
nw = nw.compute()

zeros = nw[nw.coverage == 0].count()
reaches = nw.shape[0]
nw_zeros = round(((zeros['coverage']/reaches)*100), 1)

nw = nw[nw.coverage != 0]
if nw.shape[0] > 100000:
    nw = nw.sample(100000)
nw['huc2'] = '17'

#     # Get combined reaches at 10% threshold
#     both = pd.concat(objs=[ne, sw], axis=0).reset_index()

#     combo = both.copy()
#     combo['huc2'] = 'combined'

combined = pd.concat(objs=[ne, se, sw, nw], axis=0).reset_index().drop(columns='index')

del ne, se, sw, nw

percent = str(int(float(thresholds[i])*100))

fig, ax = plt.subplots(figsize=(12,6))
sns.boxplot(data=combined, x='Bin', y='coverage',
            order=order,
            hue='huc2', palette=palette_four,
            legend=True,
            saturation=1, showmeans=True,
            fliersize=2, linewidth=2,
            boxprops=dict(alpha=0.8),
            meanprops={'marker':'o',
                       'markerfacecolor':'k',
                       'markeredgecolor':'k',
                       'markersize':'8'})

plt.title('Reaches by proportion of nodes that meet or exceed ' + percent +
          '% coverage, excluding reaches with 0% coverage\nHUC2_01: ' + str(ne_zeros) +
          '% of reaches have 0% coverage in all nodes\nHUC2_15: ' +
          str(sw_zeros) + '% of reaches have 0% coverage in all nodes')

plt.legend(title='HUC2', loc='upper right',
           title_fontproperties={'weight': 'bold'})

plt.xlabel('Width Bin [m]')
plt.ylabel('Number of nodes that exceed ' + percent + '% coverage')
plt.xticks(rotation=-45)

# plt.savefig(save_path + 'threshold_just_hucs_mean_' + percent + '.png')

### Combined

In [ ]:
for i in range(len(thresholds)):
    
    ne = ddf_ne_melted[ddf_ne_melted['threshold'] == thresholds[i]]
    ne = ne.compute().reset_index().drop(columns='index')

    zeros = ne[ne.coverage == 0].count()
    reaches = ne.shape[0]
    ne_zeros = round(((zeros['coverage']/reaches)*100), 1)
    
    ne = ne[ne.coverage != 0]
    if ne.shape[0] > 100000:
        ne = ne.sample(100000)
    
    sw = ddf_sw_melted[ddf_sw_melted['threshold'] == thresholds[i]]
    sw = sw.compute().reset_index().drop(columns='index')

    zeros = sw[sw.coverage == 0].count()
    reaches = sw.shape[0]
    sw_zeros = round(((zeros['coverage']/reaches)*100), 1)

    sw = sw[sw.coverage != 0]
    if sw.shape[0] > 100000:
        sw = sw.sample(100000)

    # Get combined reaches at 10% threshold
    combined = pd.concat(objs=[ne, sw], axis=0).reset_index()

    del ne, sw

    percent = str(int(thresholds[i]*100))

    fig, ax = plt.subplots(figsize=(10,6))
    sns.boxplot(data=combined, x='Bin', y='coverage',
                order=order,
                color=color[2],
                saturation=1, showmeans=True,
                fliersize=2, linewidth=2,
                boxprops=dict(alpha=0.8),
                meanprops={'marker':'o',
                           'markerfacecolor':'k',
                           'markeredgecolor':'k',
                           'markersize':'8'})

    plt.title('Reaches by proportion of nodes that meet or exceed ' + percent +
              '% coverage, excluding reaches with 0% coverage\nHUC2_01: ' + str(ne_zeros) +
              '% of reaches have 0% coverage in all nodes\nHUC2_15: ' +
              str(sw_zeros) + '% of reaches have 0% coverage in all nodes')

    patch = mpatches.Patch(color=color[2], label='combined')
    plt.legend(handles=[patch], title='HUC2', loc='upper right',
               title_fontproperties={'weight': 'bold'})

    plt.xticks(rotation=-45)

    plt.xlabel('Width Bin [m]')
    plt.ylabel('Proportion of nodes that exceed ' + percent + '% coverage')

    plt.savefig(save_path + 'threshold_combined_mean_' + percent + '.png')

### All three

In [ ]:
for i in range(len(thresholds)):

    # Get ne reaches at threshold
    ne = ddf_ne_melted[ddf_ne_melted['threshold'] == thresholds[i]]
    ne = ne.compute().reset_index().drop(columns=['NHDPlusID', 'index'])

    zeros = ne[ne.coverage == 0].count()
    reaches = ne.shape[0]
    ne_zeros = round(((zeros['coverage']/reaches)*100), 1)

    ne = ne[ne.coverage != 0]
    if ne.shape[0] > 100000:
        ne = ne.sample(100000)
    ne['huc2'] = '01'

    # Get sw reaches
    sw = ddf_sw_melted[ddf_sw_melted['threshold'] == thresholds[i]]
    sw = sw.compute().reset_index().drop(columns=['NHDPlusID', 'index'])

    zeros = sw[sw.coverage == 0].count()
    reaches = sw.shape[0]
    sw_zeros = round(((zeros['coverage']/reaches)*100), 1)

    sw = sw[sw.coverage != 0]
    if sw.shape[0] > 100000:
        sw = sw.sample(100000)
    sw['huc2'] = '15'

    # Get combined reaches at 10% threshold
    both = pd.concat(objs=[ne, sw], axis=0).reset_index()

    combo = both.copy()
    combo['huc2'] = 'combined'

    combined = pd.concat(objs=[ne, sw, combo], axis=0).reset_index().drop(columns=['index'])

    del ne, sw, combo

    percent = str(int(thresholds[i]*100))

    fig, ax = plt.subplots(figsize=(12,6))
    sns.boxplot(data=combined, x='Bin', y='coverage',
                order=order,
                hue='huc2', palette=palette,
                legend=True,
                saturation=1, showmeans=True,
                fliersize=2, linewidth=2,
                boxprops=dict(alpha=0.8),
                meanprops={'marker':'o',
                           'markerfacecolor':'k',
                           'markeredgecolor':'k',
                           'markersize':'8'})

    plt.title('Reaches with at least ' + percent +
              '% coverage in each node, excluding reaches with 0% coverage\nHUC2_01: ' + str(ne_zeros) +
              '% of reaches have 0% coverage in all nodes\nHUC2_15: ' +
              str(sw_zeros) + '% of reaches have 0% coverage in all nodes')

    plt.legend(title='HUC2', loc='upper right',
               title_fontproperties={'weight': 'bold'})

    plt.xlabel('Width Bin [m]')
    plt.ylabel('Reaches with at least ' + percent + '% coverage')
    plt.xticks(rotation=-45)

    plt.savefig(save_path + 'threshold_all_three_mean_' + percent + '.png')

### Combined old

In [ ]:
for i in range(len(thresholds)):
    # Make combined dataframe
    ddf_combo = dd.concat(dfs=[ddf_ne_melted, ddf_sw_melted], axis=0)

    # Get combined reaches at 10% threshold
    ddf_combo = ddf_combo[(ddf_combo['threshold'] == thresholds[i])]
    combo = ddf_combo.compute().drop(columns=['NHDPlusID']).reset_index()
    combo = combo.drop(columns=['index', 'threshold'])
    combo['coverage'] = combo['coverage']*100
    # combo['huc2'] = '01, 15'

    percent = str(int(thresholds[i]*100))

    fig, ax = plt.subplots(figsize=(10,6))
    sns.boxplot(x='Bin', y='coverage',
                data=combo,
                order=order,
                color=color[2],
                saturation=1, showmeans=True,
                fliersize=2, linewidth=2,
                boxprops=dict(alpha=0.8),
                meanprops={'marker':'o',
                           'markerfacecolor':'k',
                           'markeredgecolor':'k',
                           'markersize':'8'})

    plt.title('Reaches with at least ' + percent +'% coverage in each node, HUC2_01 and HUC2_15')
    plt.xticks(rotation=-45)

    plt.xlabel('Width Bin [m]')
    plt.ylabel('Detected [%]')

    plt.savefig('./figures_for_2025_03_06/threshold_combined_mean_' + percent + '.png')

### Side-by-side old

In [ ]:
for i in range(len(thresholds)):

    # Get ne reaches at 10% threshold
    ddf_ne_thresh = ddf_ne[(ddf_ne['threshold'] == thresholds[i])]
    ne = ddf_ne_thresh.compute().drop(columns=['NHDPlusID']).reset_index()
    ne = ne.drop(columns=['index', 'threshold'])
    ne['coverage'] = ne['coverage']*100
    ne['huc2'] = '01'

    # Get sw reaches at 10% threshold
    ddf_sw_melted_thresh = ddf_sw_melted[(ddf_sw_melted['threshold'] == thresholds[i])]
    sw = ddf_sw_melted_thresh.compute().drop(columns=['NHDPlusID']).reset_index()
    sw = sw.drop(columns=['index', 'threshold'])
    sw['coverage'] = sw['coverage']*100
    sw['huc2'] = '15'

    combo = pd.concat(objs=[ne, sw], axis=0).reset_index()

    del ne, sw

    percent = str(int(thresholds[i]*100))

    fig, ax = plt.subplots(figsize=(10,6))
    sns.boxplot(x='Bin', y='coverage', hue='huc2',
                data=combo, order=order,
                palette=palette_two,
                legend=True,
                saturation=1, showmeans=True,
                fliersize=2, linewidth=2,
                boxprops=dict(alpha=0.8),
                meanprops={'marker':'o',
                           'markerfacecolor':'k',
                           'markeredgecolor':'k',
                           'markersize':'8'})

    plt.legend(title='HUC2', loc='upper right')

    plt.title('Reaches with at least ' + percent +'% coverage in each node, HUC2_01 and HUC2_15')
    plt.xlabel('Width Bin [m]')
    plt.ylabel('Detected [%]')
    plt.xticks(rotation=-45)

    plt.savefig('./figures_for_2025_03_06/threshold_huc2_01_and_huc_15_mean_' + percent + '.png')

In [ ]:
del combo